# Data Cleaning and Creating detailed defalult user Tables 

### Loans_defaulters data

In [1]:
# Import required modules and initialize SparkSession for PySpark operations
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

### Load the raw data

In [2]:
# Read raw loan defaulters CSV data with inferred schema
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [3]:
#peek into the data
loans_def_raw_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
9cb79aa7323e81be1...,2.0,0.0,0.0,0.0,0.0,0.0,11.0,null
0dd2bbc517e3c8f9e...,0.0,0.0,1.0,1.0,3.0,0.0,null,115.0
458458599d3df3bfc...,0.0,0.0,1.0,1.0,1.0,0.0,null,76.0
05ea141ec28b5c7f7...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
aac68850fdac09fd0...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
3a423e4589e89f429...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
f1efcf7dfbfef21be...,0.0,0.0,0.0,0.0,1.0,0.0,null,null
c89986155a070db2e...,1.0,0.0,0.0,0.0,1.0,15.0,5.0,null
118dc629b6e134419...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
a86fa4b7493708333...,0.0,0.0,0.0,0.0,0.0,0.0,null,null


In [4]:
# checking schema
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



### Initial Exploration

In [5]:
# Create temporary view (loan_defaulters)
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

# Check distinct delinq_2yrs values
spark.sql("select distinct(delinq_2yrs) from loan_defaulters")

delinq_2yrs
1.0
I bike to work on...
271 monthly payme...
183xx
VISA and AMEX cre...
etc. and I feel t...
AZ
017xx
923xx
446xx


In [6]:
# Count occurrences of delinq_2yrs (reveals data issues)
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+------------------+-------+
|       delinq_2yrs|  total|
+------------------+-------+
|               0.0|1838878|
|               1.0| 281335|
|               2.0|  81285|
|               3.0|  29539|
|               4.0|  13179|
|               5.0|   6599|
|               6.0|   3717|
|               7.0|   2062|
|               8.0|   1223|
|               9.0|    818|
|              10.0|    556|
|              11.0|    363|
|              12.0|    264|
|              13.0|    165|
|              14.0|    120|
|              15.0|     87|
|              null|     63|
|              16.0|     55|
|              18.0|     30|
|              17.0|     30|
|              19.0|     23|
|              20.0|     17|
|              21.0|     12|
|                CA|      8|
|                TX|      6|
|debt_consolidation|      5|
|                IL|      5|
|    small_business|      5|
|              22.0|      5|
|              24.0|      4|
|                FL|      4|
|             

### Handle Data Quality Issues

In [7]:
# Define new schema (float types)
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

# Re-read CSV with new schema
loans_def_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

# creating temp view
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

# Non Int values appeared as NULL
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)


+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       17.0|     30|
|       18.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       23.0|      2|
|       25.0|      2|
|       3.44|      2|
|       30.0|      2|
|       29.0|      2|
|       58.0|      1|
|      18.53|      1|
|       9.44|      1|
|       6.52|      1|
|       1.37|      1|
|       3.45|      1|
|       5.52|      1|
|      22.95|      1|
|      13.76|      1|
+-----------+-------+
only showing top 40 rows



- Now we have 261 nulls after changing datatype of deling_2yrs columns from string to float 

In [8]:
# Cast delinq_2yrs to integer and Fill nulls with 0
from pyspark.sql.functions import col

loans_def_processed_df = (
    loans_def_raw_df
    .withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer"))
    .fillna(0, subset=["delinq_2yrs"])
)

# creating temp view
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

# checking for null values
spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null")

count(1)
0


### Filter members with delinquencies in the past 2 years or a delinquency record

In [9]:
# checking for active deliquent defaulters
loans_def_delinq_df = spark.sql("""
                                select member_id,delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) 
                                from loan_defaulters 
                                where delinq_2yrs > 0 
                                or mths_since_last_delinq > 0""")

# counts
loans_def_delinq_df.count()

1106163

### Filter members with public records, bankruptcies, or inquiries in the last 6 months

In [10]:
# Filter members with public records, bankruptcies, or inquiries in the last 6 months
loans_def_records_enq_df = spark.sql("""select member_id 
                                     from loan_defaulters 
                                     where pub_rec > 0.0 
                                     or pub_rec_bankruptcies > 0.0 
                                     or inq_last_6mths > 0.0
                                     """)

# count
loans_def_records_enq_df.count()

1070125

### Saving loans_def_delinq_df

In [11]:
#parquet
loans_def_delinq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/cleaned/loans_defaulters_deling_parquet") \
.save()

### Saving loans_def_records_enq_df

In [12]:
#parquet
loans_def_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/cleaned/loans_defaulters_records_enq_parquet") \
.save()

### Casting loans_def_processed_df columns into int and fillna with 0
- pub_rec
- pub_rec_bankruptcies
- inq_last_6mths

In [13]:
loans_def_p_pub_rec_df = loans_def_processed_df.withColumn("pub_rec", col("pub_rec").cast("integer")).fillna(0, subset = ["pub_rec"])
loans_def_p_pub_rec_bankruptcies_df = loans_def_p_pub_rec_df.withColumn("pub_rec_bankruptcies", col("pub_rec_bankruptcies").cast("integer")).fillna(0, subset = ["pub_rec_bankruptcies"])
loans_def_p_inq_last_6mths_df = loans_def_p_pub_rec_bankruptcies_df.withColumn("inq_last_6mths", col("inq_last_6mths").cast("integer")).fillna(0, subset = ["inq_last_6mths"])

### Select detailed records for public records and inquiries

In [14]:
loans_def_p_inq_last_6mths_df.createOrReplaceTempView("loan_defaulters")
loans_def_detail_records_enq_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths from loan_defaulters")

In [15]:
loans_def_detail_records_enq_df

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
9cb79aa7323e81be1...,0,0,0
0dd2bbc517e3c8f9e...,1,1,3
458458599d3df3bfc...,1,1,1
05ea141ec28b5c7f7...,0,0,0
aac68850fdac09fd0...,0,0,0
3a423e4589e89f429...,0,0,0
f1efcf7dfbfef21be...,0,0,1
c89986155a070db2e...,0,0,1
118dc629b6e134419...,0,0,0
a86fa4b7493708333...,0,0,0


### Saving loans_def_detail_records_enq_df

In [16]:
#parquet
loans_def_detail_records_enq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv017499/lendingclubproject/cleaned/loans_def_detail_records_enq_df_parquet") \
.save()